<a href="https://colab.research.google.com/github/Dolj0/Twitter-Sentiment-Analysis/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import time
import numpy as np
import re
import tweepy
!pip install --upgrade git+https://github.com/tweepy/tweepy@master

def auth_credentials():
  #authenticates and creates tweepy api object
  consumerKey = #keys obtained from twitter developer account
  consumerSecret = #keys obtained from twitter developer account
  accessToken = #keys obtained from twitter developer account
  accessTokenSecret = #keys obtained from twitter developer account

  # Create the authentication object
  authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
  authenticate.set_access_token(accessToken, accessTokenSecret)
  api = tweepy.API(authenticate, wait_on_rate_limit = True)
  print("authcomplete")
  return api

def cleanTxt(text):
  #cleans tweets using
  text = re.sub(r'@[A-Za-z0-9]', '', text) # Removed @mentions
  text = re.sub(r'#', '', text) #Removing # symbol
  text = re.sub(r'RT[\s]+', '', text) # Removing reTweets
  text = re.sub(r'https?:\/\/\S+', '', text) # Remove hyperlink
  if type(text) == str:
    if text == "":
      return "Neutral"
    else:
      return text
  else:
    return "Neutral"

def extraction(api):
  #Creates and opens file
  file_name = 'newfile.csv'
  csvFile = open(file_name, 'a')
  csvWriter = csv.writer(csvFile)
  csvWriter.writerow(["Date", "Tweets"])

  #Extracts tweets
  dateTo=202103240000
  while True:
    try:  #exception handling for queery request limit
      c = tweepy.Cursor(api.search_30_day, label='30DaySearch',
                                  query='DJIA', #Or, for Amazon search: query='AMZN OR Amazon -rainforest' 
                                  fromDate=202103200000,
                                  toDate=dateTo, maxResults=100).items(100)
      for tweet in c:
        print(tweet.created_at, cleanTxt(tweet.text))
        csvWriter.writerow([tweet.created_at, cleanTxt(tweet.text)]) #writes to csv
        created = str(tweet.created_at) #converts date time to string
        dateToStr = created[0:10].replace("-","")+"0000" # saves last extracted date
        dateTo = int(dateToStr)#updates To Date
    except tweepy.TweepError as e:
      time.sleep(60*15)
      break       
        
def main():
    extraction(auth_credentials())
    
if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
from textblob import TextBlob
import nltk

def getSubjectivity(text):
  #Returns subjectivity using pretrained textBlob algorithm
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  #Returns polarity using pretrained textBlob algorithm
  return TextBlob(text).sentiment.polarity

def csv_to_dataFrame(fileName):
  fileName = fileName
  df = pd.read_csv (fileName)
  return df

def dataFrame_to_csv(df):
  outputFileName = 'OutputData.csv'
  csvFileOut = open(outputFileName,'a')
  df.to_csv('/content/OutputData.csv')

def main():
  csvFile = 'newfile.csv'
  df = csv_to_dataFrame(csvFile)
  df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
  df['Polarity'] = df['Tweets'].apply(getPolarity)
  dataFrame_to_csv(df)
    
if __name__ == "__main__":
    main()